In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Know Data


In [ ]:
import os
import zipfile
import shutil

In [ ]:
os.makedirs("/kaggle/working/train", exist_ok=True)
os.makedirs("/kaggle/working/test", exist_ok=True)

In [ ]:
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip", "r") as zip:
    zip.extractall("/kaggle/working/train")

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip", "r") as zip:
    zip.extractall("/kaggle/working/test")

In [ ]:
# /kaggle/working/train/train
# train set will divide into train and validation
train_files = os.listdir("/kaggle/working/train/train")
print(train_files[0])
print(f"no. of train images {len(train_files)}")

test_files = os.listdir("/kaggle/working/test/test1")
print(f"no. of test images {len(test_files)}")

In [ ]:
base_dir = "/kaggle/working/train/train"
cat_dir = "/kaggle/working/train/cat"
dog_dir = "/kaggle/working/train/dog"

os.makedirs(cat_dir, exist_ok=True)
os.makedirs(dog_dir, exist_ok=True)


In [ ]:
for file in os.listdir(base_dir):
    if file.startswith("cat"):
        shutil.move(os.path.join(base_dir, file), os.path.join(cat_dir, file))
    elif file.startswith("dog"):
        shutil.move(os.path.join(base_dir, file), os.path.join(dog_dir, file))


In [ ]:
# "/kaggle/working/train/"  has "cat" & "dog"
print("Cats", len(os.listdir(cat_dir)))
print("Dogs", len(os.listdir(dog_dir)))

In [ ]:
!ls -d /kaggle/working/train/*/

In [ ]:
empty_dir = "/kaggle/working/train/train/"
if os.path.exists(empty_dir):
    print(os.listdir(empty_dir)) # means it is empty

In [ ]:
if os.path.exists(empty_dir):
    shutil.rmtree(empty_dir)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

cat_samples = random.sample(os.listdir(cat_dir), 4)
dog_samples = random.sample(os.listdir(dog_dir), 4)

fig, axes = plt.subplots(2, 4, figsize=(15, 8))

for i in range(4):
    # plotting cats
    cat_path = os.path.join(cat_dir, cat_samples[i])
    cat_img = mpimg.imread(cat_path)
    axes[0, i].imshow(cat_img)
    axes[0, i].set_title(cat_samples[i])
    axes[0, i].axis('off')

    # plotting dogs
    dog_path = os.path.join(dog_dir, dog_samples[i])
    dog_img = mpimg.imread(dog_path)
    axes[1, i].imshow(dog_img)
    axes[1, i].set_title(dog_samples[i])
    axes[1, i].axis('off')

plt.tight_layout() # Prevents titles from overlapping
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

folder = "/kaggle/working/test/test1"
images = os.listdir(folder) 
images = random.sample(images, 4)

fig, axes = plt.subplots(1, 4, figsize=(15, 5))

for i, img_name in enumerate(images):
    img = mpimg.imread(os.path.join(folder, img_name))
    axes[i].imshow(img)
    axes[i].set_title(img_name)
    axes[i].axis('off')

plt.show()

In [ ]:
from PIL import Image

img = Image.open(os.path.join(cat_dir, os.listdir(cat_dir)[4]))
width, height = img.size
print(f"Width: {width}, Height: {height}")

## 2. Load Data

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.utils import image_dataset_from_directory # Highly recommanded
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224) # Standard for MobileNetV3
directory = "/kaggle/working/train/"

In [ ]:
# Load the training data
train_ds = image_dataset_from_directory(
    directory,
    shuffle=True,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE, 
    batch_size=32
)

# Load the validation data
validation_ds = image_dataset_from_directory(
    directory,
    shuffle=True,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=32
)

In [ ]:
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
print(train_ds.class_names)

## 3. Preprocess and Augment Training Data

Using `dataset.prefetch` which is a important extra step in data preprocessing. 

Using `prefetch()` prevents a memory bottleneck that can occur when reading from disk. It sets aside some data and keeps it ready for when it's needed, by creating a source dataset from your input data, applying a transformation to preprocess it, then iterating over the dataset one element at a time. Because the iteration is streaming, the data doesn't need to fit into memory.

Basically using CPU and GPU to there potential.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache()            # 1. Fetch data from Ram directly instead of disk
train_ds = train_ds.shuffle(1000)      # 2. Mix them up (1000 is the buffer size)
train_ds = train_ds.prefetch(AUTOTUNE) # 3. Prepare next batch while GPU works

In [ ]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.2))
    data_augmentation.add(RandomZoom(0.1))
    
    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()

for image, _ in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

## 4. Using MobileNetV3 for Transfer Learning 

In [ ]:
# The V3 version
preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=IMG_SHAPE,
    include_top=True,
    weights='imagenet')

base_model.summary()
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# The model is trained on on 1000 Labels we want 2

In [ ]:
def catVsdogModel(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    image_shape = image_shape + (3,)

    base_model = tf.keras.applications.MobileNetV3Large(
        input_shape=image_shape,
        include_top=False, # remove ImageNet classification head (1000 classes)
        weights='imagenet')
    
    # Freeze the base model so we don't overwrite the pre-trained weights
    base_model.trainable = False

    # Build our model
    inputs = tf.keras.Input(shape=image_shape) 

    # apply data augmentaion
    x = data_augmentation(inputs)

    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x)

    x = base_model(x, training=False) 

    # Convert the 7x7 spatial features into a single vector
    x = tfl.GlobalAveragePooling2D()(x) 

    # Add Dropout to prevent overfitting 
    x = tf.keras.layers.Dropout(0.2)(x)

    outputs = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
model = catVsdogModel(IMG_SIZE, data_augmenter())

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',     # Watch the validation error
    patience=3,             # Wait 3 epochs for improvement before quitting
    restore_best_weights=True # Keep the version of the model that performed best
)

initial_epochs = 5

history = model.fit(
    train_ds, 
    validation_data=validation_ds, 
    epochs=initial_epochs,
    callbacks=[early_stopping])

In [ ]:
print(history.history.keys())

In [ ]:
# Optional: Add a starting point to both for consistency
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

# For loss, the starting point is usually high, not 0
loss = [0.7] + history.history['loss']
val_loss = [0.7] + history.history['val_loss']

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))

# Plot Training and Validation Accuracy
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Training and Validation Accuracy')

# Plot Training and Validation Loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Binary Crossentropy')
plt.ylim([0, max(val_loss)])
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')

plt.show()

## 5. Fine-tuning the Model

You could try fine-tuning the model by re-running the optimizer in the last layers to improve accuracy. When you use a smaller learning rate, you take smaller steps to adapt it a little more closely to the new data. In transfer learning, the way you achieve this is by unfreezing the layers at the end of the network, and then re-training your model on the final layers with a very low learning rate. Adapting your learning rate to go over these layers in smaller steps can yield more fine details - and higher accuracy.

The intuition for what's happening: when the network is in its earlier stages, it trains on low-level features, like edges. In the later layers, more complex, high-level features like wispy hair or pointy ears begin to emerge. For transfer learning, the low-level features can be kept the same, as they have common features for most images. When you add new data, you generally want the high-level features to adapt to it, which is rather like letting the network learn to detect features more related to your data, such as soft fur or big teeth. 

To achieve this, just unfreeze the final layers and re-run the optimizer with a smaller learning rate, while keeping all the other layers frozen.

Where the final layers actually begin is a bit arbitrary, so feel free to play around with this number a bit. The important takeaway is that the later layers are the part of your network that contain the fine details (pointy ears, hairy tails) that are more specific to your problem.

First, unfreeze the base model by setting `base_model.trainable=True`, set a layer to fine-tune from, then re-freeze all the layers before it. Run it again for another few epochs, and see if your accuracy improved!

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [ ]:
base_model = model.get_layer('MobileNetV3Large')
base_model.trainable = True

# Freeze everything except the last 30 layers
fine_tune_at = 165

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Quick check to make sure it worked
trainable_count = len([l for l in base_model.layers if l.trainable])
print(f"Base model layers: {len(base_model.layers)}")
print(f"Layers now trainable: {trainable_count}")

In [ ]:
loss_function= tf.keras.losses.BinaryCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5) # 0.00001
metrics=['accuracy']

model.compile(loss=loss_function,
              optimizer = optimizer,
              metrics=metrics)

In [ ]:
model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=3,             # Stop if val_loss doesn't improve for 3 epochs
    restore_best_weights=True # Very important: rolls back to the best version
)

fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs # current_epochs + 10

history_fine = model.fit(
    train_ds,
    epochs=total_epochs,     
    initial_epoch=history.epoch[-1],
    validation_data=validation_ds,
    callbacks=[early_stop]   
)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.evaluate(validation_ds)

In [ ]:
import matplotlib.image as mpimg
import numpy as np
import os
import random
import tensorflow as tf

folder = "/kaggle/working/test/test1"
images = os.listdir(folder) 
images = random.sample(images, 4)

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Get class names (usually ['cat', 'dog']) from your training dataset
# If you don't have it, we assume 0=Cat, 1=Dog based on alphabetical order
class_names = ['Cat', 'Dog'] 

for i, img_name in enumerate(images):
    img_path = os.path.join(folder, img_name)
    
    # 1. Load and Preprocess the image for the model
    # It must be the same size used during training (224x224)
    img_load = tf.keras.utils.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img_load)
    img_array = tf.expand_dims(img_array, 0) # Create a batch of 1
    
    # 2. Make Prediction
    prediction = model.predict(img_array, verbose=0)
    score = prediction[0][0] # Since it's a sigmoid, it returns a single value
    
    # 3. Determine Label and Confidence
    # If score > 0.5 it's a Dog (1), otherwise it's a Cat (0)
    if score > 0.5:
        label = "Dog"
        confidence = score * 100
    else:
        label = "Cat"
        confidence = (1 - score) * 100

    # 4. Show the actual image
    img_display = mpimg.imread(img_path)
    axes[i].imshow(img_display)
    axes[i].set_title(f"Pred: {label}\nConf: {confidence:.2f}%")
    axes[i].axis('off')

plt.show()

In [ ]:
model.save("/kaggle/working/cat_vs_dog_model.keras")

## 6. Prediction

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 1. Load the model (this reconstructs everything from the file)
trained_model_path = "/kaggle/input/dogvscat-v1/keras/default/1/cat_vs_dog_model.keras"
trained_model = tf.keras.models.load_model(model_path)

trained_model.evaluate(validation_ds)

In [ ]:
test_dir = "/kaggle/working/test/test1"

test_img = os.listdir(test_dir)[1]
test_img_path = os.path.join(test_dir, test_img)

img = tf.keras.utils.load_img(test_img_path, target_size=(224, 224))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Model expects a batch (1, 224, 224, 3)

logit = model.predict(img_array)
print(logit)

In [ ]:
probability = tf.nn.sigmoid(logit).numpy()[0][0]
print(probability)

In [ ]:
if probability > 0.5:
    print(f"Prediction: DOG ({probability:.2%})")
else:
    print(f"Prediction: CAT ({1 - probability:.2%})")

In [ ]:
img = tf.keras.utils.load_img(test_img_path)
img = tf.keras.utils.img_to_array(img).astype("uint8")

plt.imshow(img)
plt.axis('off')
plt.show()


## 7. Converting to TensorFlow Lite

If you use models architecture build during traing which has data augmentation layer it will cause problem during tensorflow lite conversion cause it introduces randomness to data which was good during trainign but not during inference.

In [ ]:
trained_model.summary()

In [ ]:
for i, layer in enumerate(trained_model.layers):
    print(f"Layer {i}: {layer.name}")

# we dont need that Layer 1 i.e Data augmentaion layer 

In [ ]:
# the .get_layer(), uses the same layer objects (already trained ).
mobilenet = trained_model.get_layer("MobileNetV3Large")
gap = trained_model.get_layer("global_average_pooling2d_1")
dropout = trained_model.get_layer("dropout_1")
classifier = trained_model.get_layer("dense")

In [ ]:
IMG_SIZE = (224, 224)
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))

x = mobilenet(inputs)
x = gap(x)
x = dropout(x, training=False)   # Dropout OFF for inference
outputs = classifier(x)

inference_model = tf.keras.Model(inputs, outputs) 

In [ ]:
inference_model.summary()

In [ ]:
dir_path = '/kaggle/working/saved_model'

if os.path.exists(dir_path):
    shutil.rmtree(dir_path) 


inference_model.export("saved_model")

In [ ]:
loaded = tf.saved_model.load("/kaggle/working/saved_model")

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

In [ ]:
def representative_data_gen():
    for images, _ in train_ds.take(100):
        yield [images]

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("/kaggle/working/saved_model")

# Enable optimization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Calibration data
converter.representative_dataset = representative_data_gen

# Force full INT8
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]

# Input / output also int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [ ]:
tflite_model = converter.convert()

In [ ]:
with open("model_int8.tflite", "wb") as f:
    f.write(tflite_model)